In [21]:
;; Core Data Structures (using closures for encapsulation)

;; Configuration builder - uses closure to accumulate state
(define (make-config-builder)
  (let ((grid-cfg '())
        (functions '())
        (kernels '())
        (lifeforms '())
        (simulation '())
        (display-cfg '()))
    (lambda (msg . args)
      (cond
        ((eq? msg 'set-grid!) (set! grid-cfg (car args)))
        ((eq? msg 'add-function!) (set! functions (cons (car args) functions)))
        ((eq? msg 'add-kernel!) (set! kernels (cons (car args) kernels)))
        ((eq? msg 'add-lifeform!) (set! lifeforms (cons (car args) lifeforms)))
        ((eq? msg 'set-simulation!) (set! simulation (car args)))
        ((eq? msg 'set-display!) (set! display-cfg (car args)))
        ((eq? msg 'build)
         (list (cons 'grid grid-cfg)
               (cons 'functions (reverse functions))
               (cons 'kernels (reverse kernels))
               (cons 'lifeforms (reverse lifeforms))
               (cons 'simulation simulation)
               (cons 'display display-cfg)))
        (else '())))))

(define (make-pair . xs) (cons (car xs) (cadr xs)))
(define (alist-ref al k)
  (let loop ((a al))
    (if (null? a)
        #f
        (if (eq? (caar a) k) (cdar a) (loop (cdr a))))))

(define (ensure-symbol x) (if (symbol? x) x (string->symbol x)))
(define (ensure-string x) (if (string? x) x (symbol->string x)))

In [22]:
;; Hygienic Macros for Declarative Syntax

;; Main configuration macro - establishes the automaton context
(define-syntax automaton
  (syntax-rules (grid functions kernels lifeforms simulation display)
    ((_ (grid w h)
        (functions f ...)
        (kernels k ...)
        (lifeforms l ...)
        (simulation s ...)
        (display d ...))
     (let ((b (make-config-builder)))
       (begin
         (b 'set-grid! (list (make-pair 'width w) (make-pair 'height h)))
         (begin (b 'add-function! f) ...)
         (begin (b 'add-kernel! k) ...)
         (begin (b 'add-lifeform! l) ...)
         (b 'set-simulation! (list s ...))
         (b 'set-display! (list d ...))
         (b 'build))))))

In [23]:
;; Function definition macro with pattern matching

(define-syntax define-function
  (syntax-rules (gaussian step lenia regression sigmoid tanh mean: 
                 sigma: growth: threshold: slope: mu: beta: a: b: c: 
                 k: x0: scale: bias:)

    ((_ name (gaussian mean: m sigma: s growth: g))
     (list (quote name) 'gaussian m s g))
    ((_ name (step threshold: t slope: a))
     (list (quote name) 'step t a))
    ((_ name (lenia mu: m sigma: s beta: b))
     (list (quote name) 'lenia m s b))
    ((_ name (regression a: a b: b c: c))
     (list (quote name) 'regression a b c))
    ((_ name (sigmoid k: k x0: x0))
     (list (quote name) 'sigmoid k x0))
    ((_ name (tanh scale: s bias: b))
     (list (quote name) 'tanh s b))
    ((_ name (conway underpopulation: u overpopulation: o reproduction: r))
     (list (quote name) 'conway u o r))))

In [24]:
;; Kernel definition macro
;; - aceita disk, ring, square
;; - para square, aceita custom_array como string: e.g. (square custom_array: "[[1,1,1],[1,0,1],[1,1,1]]")
(define-syntax define-kernel
  (syntax-rules (disk ring square blur: outer: inner: side: sigma: size: custom: radius: custom_array:)
    ;; disk: (define-kernel name (disk radius: r blur: (sigma: s size: sz)))
    ((_ name (disk radius: r blur: (sigma: s size: sz)))
     (list (quote name) 'disk r s sz))
    ;; ring: (define-kernel name (ring outer: out inner: in blur: (sigma: s size: sz)))
    ((_ name (ring outer: out inner: in blur: (sigma: s size: sz)))
     (list (quote name) 'ring out in s sz))
    ;; square without custom array
    ((_ name (square side: side blur: (sigma: s size: sz) custom: arr))
     (list (quote name) 'square side s sz arr))
    ((_ name (square side: side blur: (sigma: s size: sz)))
     (list (quote name) 'square side s sz #f))
    ;; square WITH custom array passed as a string literal
    ((_ name (square custom_array: s))
     (list (quote name) 'square #f #f #f s))
    ;; square WITH custom array and explicit side/blur: (optional)
    ((_ name (square side: side blur: (sigma: s size: sz) custom: arr))
     (list (quote name) 'square side s sz arr))))


In [25]:
; Not tested High order kernel operations

;; Kernel combinator - creates composite kernels using operations
(define (kernel-combinator op)
  (lambda (k1 k2)
    (list 'composite op k1 k2)))

;; Specific kernel operators (curried combinators)
(define kernel+ (kernel-combinator 'add))
(define kernel- (kernel-combinator 'subtract))
(define kernel* (kernel-combinator 'multiply))
(define kernel/ (kernel-combinator 'divide))

;; Scale kernel by constant (higher-order function)
(define (scale-kernel factor)
  (lambda (kernel)
    (kernel* kernel factor)))

;; Kernel reference (for reusing named kernels)
(define (kernel-ref name)
  (list 'ref name))

;; Function reference (for reusing named functions)
(define (function-ref name)
  (list 'ref name))

In [26]:
;; Lifeform definition macro
(define-syntax define-lifeform
  (syntax-rules (color: initial: rules:)
    ((_ name (color: col) (initial: init) (rules: r ...))
     (list (quote name) (ensure-string col) (quote init) (list r ...)))))

In [27]:
(define-syntax rule
  (syntax-rules (-> dt: kernel: function: weight:)
    ((_ target -> (dt: dtv kernel: k function: f weight: w))
     (list (quote target) dtv (quote k) (quote f) w))))

In [28]:
(define-syntax dt
  (syntax-rules ()
    ((_ v) (cons 'dt v))))
(define-syntax steps
  (syntax-rules ()
    ((_ v) (cons 'steps v))))
(define-syntax seed
  (syntax-rules ()
    ((_ v) (cons 'seed v))))
(define-syntax diffusion
  (syntax-rules ()
    ((_ v) (cons 'diffusion v))))

(define-syntax window
  (syntax-rules ()
    ((_ v) (cons 'window (quote v)))))
(define-syntax fps
  (syntax-rules ()
    ((_ v) (cons 'fps v))))
(define-syntax quit
  (syntax-rules ()
    ((_ v) (cons 'quit (quote v)))))
(define-syntax scale
  (syntax-rules ()
    ((_ v) (cons 'scale v))))

In [29]:
(define (yaml-str . xs) (apply string-append xs))
(define (yaml-num n)
  (if (or (not n) (eq? n #f))
      "" ; ou "null", dependendo do que o YAML espera
      (number->string n)))
(define (yaml-sym s) (symbol->string s))

(define (yaml-indent n) (make-string n #\space))
(define (yaml-kv indent key val)
  (yaml-str (yaml-indent indent) (ensure-string key) ": " val "\n"))

Agora, uma série de funções tediosas que servem para escrever de maneira formatada o texto do arquivo .yml 

In [30]:
(define (yaml-write-func-params type params)
  (cond
    ((eq? type 'gaussian)
     (yaml-str "    mean: " (yaml-num (list-ref params 0)) "\n"
               "    sigma: " (yaml-num (list-ref params 1)) "\n"
               "    growth: " (yaml-num (list-ref params 2)) "\n"))
    ((eq? type 'step)
     (yaml-str "    threshold: " (yaml-num (list-ref params 0)) "\n"
               "    slope: " (yaml-num (list-ref params 1)) "\n"))
    ((eq? type 'lenia)
     (yaml-str "    mu: " (yaml-num (list-ref params 0)) "\n"
               "    sigma: " (yaml-num (list-ref params 1)) "\n"
               "    beta: " (yaml-num (list-ref params 2)) "\n"))
    ((eq? type 'regression)
     (yaml-str "    a: " (yaml-num (list-ref params 0)) "\n"
               "    b: " (yaml-num (list-ref params 1)) "\n"
               "    c: " (yaml-num (list-ref params 2)) "\n"))
    ((eq? type 'sigmoid)
     (yaml-str "    k: " (yaml-num (list-ref params 0)) "\n"
               "    x0: " (yaml-num (list-ref params 1)) "\n"))
    ((eq? type 'tanh)
     (yaml-str "    scale: " (yaml-num (list-ref params 0)) "\n"
               "    bias: " (yaml-num (list-ref params 1)) "\n"))
    (else "")))

(define (yaml-write-functions funcs)
  (if (null? funcs)
      ""
      (apply string-append
             (map (lambda (f)
                    (let ((name (list-ref f 0))
                          (type (list-ref f 1))
                          (params (cddr f)))
                      (yaml-str "  " (yaml-sym name) ":\n"
                                "    type: " (yaml-sym type) "\n"
                                (yaml-write-func-params type params))))
                  funcs))))

In [31]:
(define (yaml-write-kernel-body type params)
  (cond
    ((eq? type 'disk)
     (yaml-str "    type: disk\n"
               "    radius: " (yaml-num (list-ref params 0)) "\n"
               "    gaussian_sigma: " (yaml-num (list-ref params 1)) "\n"
               "    gaussian_kernel_size: " (yaml-num (list-ref params 2)) "\n"))
    ((eq? type 'ring)
     (yaml-str "    type: ring\n"
               "    outer_diameter: " (yaml-num (list-ref params 0)) "\n"
               "    inner_diameter: " (yaml-num (list-ref params 1)) "\n"
               "    gaussian_sigma: " (yaml-num (list-ref params 2)) "\n"
               "    gaussian_kernel_size: " (yaml-num (list-ref params 3)) "\n"))
    ((eq? type 'square)
     (let ((side (list-ref params 0))
           (sigma (list-ref params 1))
           (ksz (list-ref params 2))
           (arr (list-ref params 3)))
       (yaml-str "    type: square\n"
                 "    side: " (yaml-num side) "\n"
                 "    gaussian_sigma: " (yaml-num sigma) "\n"
                 "    gaussian_kernel_size: " (yaml-num ksz) "\n"
                 (if arr (yaml-str "    custom_array: " (ensure-string arr) "\n") ""))))
    (else "")))

(define (yaml-write-kernels kernels)
  (if (null? kernels)
      ""
      (apply string-append
             (map (lambda (k)
                    (let ((name (list-ref k 0))
                          (type (list-ref k 1))
                          (params (cddr k)))
                      (yaml-str "  " (yaml-sym name) ":\n"
                                (yaml-write-kernel-body type params))))
                  kernels))))

In [32]:
(define (yaml-write-rule r)
  (let ((name (list-ref r 0))
        (dt (list-ref r 1))
        (kern (list-ref r 2))
        (func (list-ref r 3))
        (weight (list-ref r 4)))
    (yaml-str "      " (yaml-sym name) ":\n"
              "        dt: " (yaml-num dt) "\n"
              "        kernel:\n"
              "          ref: " (yaml-sym kern) "\n"
              "        func:\n"
              "          ref: " (yaml-sym func) "\n"
              "        weight: " (yaml-num weight) "\n")))

(define (yaml-write-rules rules)
  (apply string-append (map yaml-write-rule rules)))

(define (yaml-write-lifeforms lifes)
  (if (null? lifes)
      ""
      (apply string-append
             (map (lambda (life)
                    (let ((name (list-ref life 0))
                          (color (list-ref life 1))
                          (initial (list-ref life 2))
                          (rules (list-ref life 3)))
                      (yaml-str "  " (yaml-sym name) ":\n"
                                "    color: \"" color "\"\n"
                                "    initial_state: " (yaml-sym initial) "\n"
                                "    rules:\n"
                                (yaml-write-rules rules)
                                "\n")))
                  lifes))))

In [33]:
(define (yaml-write-grid grid)
  (if (null? grid)
      ""
      (let ((w (cdr (assq 'width grid)))
            (h (cdr (assq 'height grid))))
        (yaml-str "  width: " (yaml-num w) "\n"
                  "  height: " (yaml-num h) "\n"))))

In [34]:
(define (yaml-write-simulation sim)
  (if (null? sim)
      ""
      (let loop ((rest sim) (acc ""))
        (if (null? rest)
            acc
            (let* ((p (car rest))
                   (k (car p))
                   (v (cdr p)))
              (loop (cdr rest)
                    (string-append acc
                      (case k
                        ((dt) (yaml-kv 2 "dt" (yaml-num v)))
                        ((steps) (yaml-kv 2 "steps" (yaml-num v)))
                        ((seed) (yaml-kv 2 "seed" (yaml-num v)))
                        ((diffusion) (yaml-kv 2 "diffusion" (yaml-num v)))
                        (else (yaml-kv 2 (yaml-sym k) (ensure-string v)))))))))))

In [35]:
(define (yaml-write-display d)
  (if (null? d)
      ""
      (let loop ((rest d) (acc ""))
        (if (null? rest)
            acc
            (let* ((p (car rest))
                   (k (car p))
                   (v (cdr p)))
              (loop (cdr rest)
                    (string-append acc
                      (case k
                        ((window) (yaml-kv 2 "window_name" (yaml-sym v)))
                        ((fps)
                            (let ((n (if (list? v) (car v) v)))
                                (yaml-kv 2 "show_fps" (if (> n 0) "true" "false"))))
                        ((quit) (yaml-kv 2 "quit_key" (yaml-sym v)))
                        (else (yaml-kv 2 (yaml-sym k) (ensure-string v)))))))))))

In [36]:
(define (yaml-write-top-section p)
  (let ((section (car p))
        (data (cdr p)))
    (cond
      ((eq? section 'grid) (string-append "grid:\n" (yaml-write-grid data)))
      ((eq? section 'functions) (string-append "functions:\n" (yaml-write-functions data)))
      ((eq? section 'kernels) (string-append "kernels:\n" (yaml-write-kernels data)))
      ((eq? section 'lifeforms) (string-append "lifeforms:\n" (yaml-write-lifeforms data)))
      ((eq? section 'simulation) (string-append "simulation:\n" (yaml-write-simulation data)))
      ((eq? section 'display) (string-append "display:\n" (yaml-write-display data)))
      (else ""))))

(define (config->yaml config)
  (apply string-append (map yaml-write-top-section config)))

In [37]:
(define (show-config cfg)
  (display (config->yaml cfg)))

(display "DSL loaded.\n")

DSL loaded.


In [38]:
; Teste de escrita de configuração

(define cfg
  (automaton
    (grid 64 64)
    (functions
      (define-function f1 (gaussian mean: 0.5 sigma: 0.15 growth: 1.0))
      (define-function step1 (step threshold: 0.4 slope: 12)))
    (kernels
      (define-kernel k1 (disk radius: 3 blur: (sigma: 1 size: 5)))
      (define-kernel k2 (ring outer: 5 inner: 2 blur: (sigma: 1 size: 5))))
    (lifeforms
      (define-lifeform amoeba
        (color: "green")
        (initial: random)
        (rules:
          (rule A -> (dt: 0.05 kernel: k1 function: f1 weight: 1.0))
          (rule B -> (dt: 0.05 kernel: k2 function: step1 weight: 0.6)))))
    (simulation
      (dt 0.05)
      (steps 1000)
      (seed 42))
    (display
      (window main)
      (fps 60)
      (quit q))))

(show-config cfg)

grid:
  width: 64
  height: 64
functions:
  f1:
    type: gaussian
    mean: 0.5
    sigma: 0.15
    growth: 1.0
  step1:
    type: step
    threshold: 0.4
    slope: 12
kernels:
  k1:
    type: disk
    radius: 3
    gaussian_sigma: 1
    gaussian_kernel_size: 5
  k2:
    type: ring
    outer_diameter: 5
    inner_diameter: 2
    gaussian_sigma: 1
    gaussian_kernel_size: 5
lifeforms:
  amoeba:
    color: "green"
    initial_state: random
    rules:
      A:
        dt: 0.05
        kernel:
          ref: k1
        func:
          ref: f1
        weight: 1.0
      B:
        dt: 0.05
        kernel:
          ref: k2
        func:
          ref: step1
        weight: 0.6

simulation:
  dt: 0.05
  steps: 1000
  seed: 42
display:
  window_name: main
  show_fps: true
  quit_key: q


In [ ]:
; Exemplo 1: Duas formas de vida interagindo

(define cfg
  (automaton
    (grid 128 128)
    (functions
      (define-function f1 (gaussian mean: 0.6 sigma: 0.1 growth: 1.2))
      (define-function f2 (step threshold: 0.3 slope: 10)))
    (kernels
      (define-kernel k1 (square side: 3 blur: (sigma: 1 size: 3)))
      (define-kernel k2 (disk radius: 4 blur: (sigma: 0.5 size: 7))))
    (lifeforms
      (define-lifeform predator
        (color: "red")
        (initial: random_discrete)
        (rules:
          (rule hunt -> (dt: 0.1 kernel: k1 function: f1 weight: 1.0))))
      (define-lifeform prey
        (color: "blue")
        (initial: random_discrete)
        (rules:
          (rule escape -> (dt: 0.1 kernel: k2 function: f2 weight: 0.8)))))
    (simulation
      (dt 0.1)
      (steps 500)
      (seed 123))
    (display
      (window main)
      (fps 30)
      (quit q))))

(show-config cfg)


grid:
  width: 128
  height: 128
functions:
  f1:
    type: gaussian
    mean: 0.6
    sigma: 0.1
    growth: 1.2
  f2:
    type: step
    threshold: 0.3
    slope: 10
kernels:
  k1:
    type: square
    side: 3
    gaussian_sigma: 1
    gaussian_kernel_size: 3
  k2:
    type: disk
    radius: 4
    gaussian_sigma: 0.5
    gaussian_kernel_size: 7
lifeforms:
  predator:
    color: "red"
    initial_state: random_discrete
    rules:
      hunt:
        dt: 0.1
        kernel:
          ref: k1
        func:
          ref: f1
        weight: 1.0

  prey:
    color: "blue"
    initial_state: random_discrete
    rules:
      escape:
        dt: 0.1
        kernel:
          ref: k2
        func:
          ref: f2
        weight: 0.8

simulation:
  dt: 0.1
  steps: 500
  seed: 123
display:
  window_name: main
  show_fps: true
  quit_key: q


In [ ]:
; Exemplo 2: Uma forma de vida com múltiplas regras

(define cfg
  (automaton
    (grid 100 100)
    (functions
      (define-function f1 (gaussian mean: 0.5 sigma: 0.2 growth: 1.0))
      (define-function f2 (step threshold: 0.7 slope: 15)))
    (kernels
      (define-kernel k1 (disk radius: 2 blur: (sigma: 1 size: 5)))
      (define-kernel k2 (ring outer: 5 inner: 1 blur: (sigma: 0.8 size: 5))))
    (lifeforms
      (define-lifeform slime
        (color: "purple")
        (initial: random_discrete)
        (rules:
          (rule spread -> (dt: 0.05 kernel: k1 function: f1 weight: 1.0))
          (rule retreat -> (dt: 0.05 kernel: k2 function: f2 weight: 0.5)))))
    (simulation
      (dt 0.05)
      (steps 800)
      (seed 99))
    (display
      (window main)
      (fps 60)
      (quit q))))

(show-config cfg)

grid:
  width: 100
  height: 100
functions:
  f1:
    type: gaussian
    mean: 0.5
    sigma: 0.2
    growth: 1.0
  f2:
    type: step
    threshold: 0.7
    slope: 15
kernels:
  k1:
    type: disk
    radius: 2
    gaussian_sigma: 1
    gaussian_kernel_size: 5
  k2:
    type: ring
    outer_diameter: 5
    inner_diameter: 1
    gaussian_sigma: 0.8
    gaussian_kernel_size: 5
lifeforms:
  slime:
    color: "purple"
    initial_state: random_discrete
    rules:
      spread:
        dt: 0.05
        kernel:
          ref: k1
        func:
          ref: f1
        weight: 1.0
      retreat:
        dt: 0.05
        kernel:
          ref: k2
        func:
          ref: f2
        weight: 0.5

simulation:
  dt: 0.05
  steps: 800
  seed: 99
display:
  window_name: main
  show_fps: true
  quit_key: q


In [ ]:
; exemplo 3: Variante de Conway com kernel customizado

(define cfg
  (automaton
    (grid 64 64)
    (functions
      (define-function conway_rules (conway underpopulation: 2 overpopulation: 3 reproduction: 3)))
    (kernels
      (define-kernel conway_kernel (square custom_array: "[[1 1 1] [1 0 1] [1 1 1]]")))
    (lifeforms
      (define-lifeform conway
        (color: "red")
        (initial: random_discrete)
        (rules:
          (rule conway -> (dt: 1.0 kernel: conway_kernel function: conway_rules weight: 1.0))))
      (define-lifeform conway2
        (color: "blue")
        (initial: random_discrete)
        (rules:
          (rule conway2 -> (dt: 1.0 kernel: conway_kernel function: conway_rules weight: 1.0)))))
    (simulation
      (dt 1.0)
      (steps 500)
      (seed 42))
    (display
      (window main)
      (fps 30)
      (quit q))))

(show-config cfg)


grid:
  width: 64
  height: 64
functions:
  conway_rules:
    type: conway
kernels:
  conway_kernel:
    type: square
    side: 
    gaussian_sigma: 
    gaussian_kernel_size: 
    custom_array: [[1 1 1] [1 0 1] [1 1 1]]
lifeforms:
  conway:
    color: "red"
    initial_state: random_discrete
    rules:
      conway:
        dt: 1.0
        kernel:
          ref: conway_kernel
        func:
          ref: conway_rules
        weight: 1.0

  conway2:
    color: "blue"
    initial_state: random_discrete
    rules:
      conway2:
        dt: 1.0
        kernel:
          ref: conway_kernel
        func:
          ref: conway_rules
        weight: 1.0

simulation:
  dt: 1.0
  steps: 500
  seed: 42
display:
  window_name: main
  show_fps: true
  quit_key: q
